In [ ]:
import scipy.io
import numpy as np
import os
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt

In [ ]:
# INPUT:
directory = './calib_data_4'
image_id = [0,1,2,3,4,5,6,7,8,9,10,12,13,14,16,17,18,19]

# Camera intrinsics
K = np.eye(3)
K[0,0] = 1.748420877529604e+03
K[1,1] = 1.749299610670773e+03
K[0,2] = 7.120351208245908e+02
K[1,2] = 4.953153651767082e+02

# Camera distortion (5,)
distortion = (-0.282673608453370, 0.197356521183472,0,0,0)

# ChessBoard params
board_size =  (11, 9)
square_size = 0.0424 # m

save_figures = False

# Camera pose from end-effector frame
camera_translation = np.array([[0.337760678839, -0.46480742266, 0.0250792296204]])
camera_rotation = R.from_quat([-0.513693426639, 0.496370265856, 0.49749117207, 0.492176956302])
T_camera_effector = np.eye(4)
T_camera_effector[:3,3] = camera_translation
T_camera_effector[:3,:3] = camera_rotation.as_matrix()
print(T_camera_effector)


In [ ]:
def transformation_from_rotation_translation(rotation, translation):
    transformations = []
    for r, t in zip(rotation,translation):
        T = np.eye(4)
        T[:3, :3] = r
        T[:3, 3] = t
        transformations.append(T)
    return np.array(transformations)

def inv(T):
    T_inv = np.eye(4)
    T_inv[:3,:3] = np.linalg.inv(T[:3,:3])
    T_inv[:3,3] = -np.matmul(np.linalg.inv(T[:3,:3]), T[:3,3]) 
    return T_inv    

In [ ]:
from matplotlib.patches import FancyArrowPatch

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)

def insert_frame(T, scale, ax):
    o = np.array([0,0,0,1])
    x = np.array([scale,0,0,1])
    y = np.array([0,scale,0,1])
    z = np.array([0,0,scale,1])
    o_w = np.matmul(T, o)
    x_w = np.matmul(T, x)
    y_w = np.matmul(T, y)
    z_w = np.matmul(T, z)
    # Here we create the arrows:
    arrow_prop_dict = dict(mutation_scale=20, arrowstyle='->', shrinkA=0, shrinkB=0)
    a = Arrow3D([o_w[0], x_w[0]], [o_w[1], x_w[1]], [o_w[2], x_w[2]], **arrow_prop_dict, color='r')
    ax.add_artist(a)
    a = Arrow3D([o_w[0], y_w[0]], [o_w[1], y_w[1]], [o_w[2], y_w[2]], **arrow_prop_dict, color='g')
    ax.add_artist(a)
    a = Arrow3D([o_w[0], z_w[0]], [o_w[1], z_w[1]], [o_w[2], z_w[2]], **arrow_prop_dict, color='b')
    ax.add_artist(a)




In [ ]:
# End-effector pose from robot base/world frame

# Load invidual text files with (x,y,z,qx,qy,qz,qw) end-effector poses
robot_pose = []
for i in image_id:
    filename = 'robot_pose/{}_robot_pose.txt'.format(i)
    try:
        robot_pose.append(np.loadtxt(os.path.join(directory, filename), delimiter=','))
    except:
        pass
robot_pose = np.array(robot_pose)

# Save single text file with all relevant robot poses for visp_handeye_calibrator 
np.savetxt(os.path.join(directory, 'effector2world_transform.txt'), robot_pose, delimiter=', ', fmt='%f')

# Construct Nx4x4 matrix containing homogenous transformation matrices for reprojection purposes
robot_translation = []
robot_rotation = []
for pose in robot_pose:
    robot_translation.append(pose[:3])
    r = R.from_quat(pose[3:])
    robot_rotation.append(r.as_matrix())
robot_translation = np.array(robot_translation)
robot_rotation = np.array(robot_rotation)
T_effector_world = transformation_from_rotation_translation(robot_rotation, robot_translation)
print(T_effector_world.shape)


In [ ]:
# Pattern pose from camera (Extrinsics)
# Calibration result loaded from Matlab cameraCalibrator generated script
calib_result = scipy.io.loadmat('calib_result') 
pattern_rotation = calib_result['rotation'].transpose(2,0,1)
pattern_translation = calib_result['translation']/1000.0

# Construct Nx4x4 homogenious transformation matrices for reprojetion
T_pattern_camera = transformation_from_rotation_translation(pattern_rotation, pattern_translation)
print(T_pattern_camera.shape)

# Convert to quaternions and save to single text file for visp_handeye_cablibrator
for i in range(len(pattern_rotation)):
    pattern_rotation[i] = pattern_rotation[i].T    
r = R.from_matrix(pattern_rotation)
quat = r.as_quat()
object2cam_transform = np.concatenate((pattern_translation.T, quat.T), axis=0)
np.savetxt(os.path.join(directory, 'object2cam_transform.txt'), object2cam_transform.T, delimiter=', ', fmt='%f')


In [ ]:
imagePoints = scipy.io.loadmat('imagePoints')['imagePoints']

# Chess coordinates in chess frame
point_coordinates = []
for x in range(board_size[0]):
    for y in range(board_size[1]):
        point_coordinates.append([x*square_size, y*square_size, 0, 1])
points = np.array(point_coordinates).T

# Homogenious transformation matrices
# T_cb: board in camera frame (Nx4x4)
# T_ec: camera in end-effector frame (4x4)
# T_we: end-effector in world frame (Nx4x4)

T_cb = np.array(T_pattern_camera)
T_ec = np.array(T_camera_effector)
T_we = np.array(T_effector_world)
for i in range(0,len(image_id)):
    T_cb[i][:3,:3] = T_cb[i][:3,:3].T

imgs = []
for i in image_id:
    imgs.append(cv2.imread(os.path.join(directory, 'images/{}.jpg'.format(i))))

# Camera projection matrix: P=K[R|t]
Rt = np.array([[1,0,0,0],
               [0,1,0,0],
               [0,0,1,0]])
P = np.matmul(K,Rt)

for i in range(0,len(image_id)):
    # undistort image
    undistorted_img = cv2.undistort(imgs[i], K, distortion, None, K)
    undistorted_img = cv2.cvtColor(undistorted_img, cv2.COLOR_BGR2RGB)
    # undistort points detected in chessboard
    image_points = cv2.undistortPoints(imagePoints[:,:,i], K, distortion)
    image_points = image_points.transpose(2,1,0).reshape(2,-1) 
    # applying necessary fix to account for undistortion
    image_points[0] = image_points[0]*K[0,0]+K[0,2]
    image_points[1] = image_points[1]*K[1,1]+K[1,2] 
    for j in range(0,len(image_id)):
        # Points in j'th board frame transformed and reprojected into i'th image 
        points_j_in_camera_j = np.matmul(T_cb[j], points)
        points_j_in_ee_j = np.matmul(T_ec, points_j_in_camera_j)
        points_j_in_world = np.matmul(T_we[j], points_j_in_ee_j)
        points_j_in_ee_i = np.matmul(inv(T_we[i]), points_j_in_world)
        points_j_in_camera_i = np.matmul(inv(T_ec), points_j_in_ee_i)
        
        reprojection = np.matmul(P, points_j_in_camera_i)
        reprojection[0]/=reprojection[2] 
        reprojection[1]/=reprojection[2] 
        reprojection[2]/=reprojection[2] 
        plt.figure(figsize=(15,15))
        plt.scatter(image_points[0], image_points[1], marker='o', s=80, linewidths=1, facecolors='none', edgecolors='g')
        plt.plot(reprojection[0],reprojection[1], 'rx', markersize=8)
        plt.imshow(undistorted_img)
        if save_figures:
            plt.savefig(os.path.join(directory, 'reprojection_error/{}_{}.png'.format(i,j)))
        plt.show()
        
  


In [ ]:
# 3D reprojection
%matplotlib
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_xlim(0, 2)
ax.set_ylabel('Y')
ax.set_ylim(-1, 1)
ax.set_zlabel('Z')
ax.set_zlim(-1, 1)
insert_frame(np.eye(4), scale=1, ax=ax) # plot world frame
n = 20

T_cbs = np.array(T_pattern_camera)
T_ec = np.array(T_camera_effector)
T_wes = np.array(T_effector_world)
for i in range(0,len(image_id)):
    T_cbs[i][:3,:3] = T_cbs[i][:3,:3].T
    
t = []
for T_we, T_cb in zip(T_wes,T_cbs):
    #insert_frame(T_we, scale=0.2, ax=ax) # plot end-effector frame 
    insert_frame(np.matmul(T_we, T_ec), scale=0.4, ax=ax) # plot camera frame
    insert_frame(np.matmul(T_we, np.matmul(T_ec, T_cb)), scale=0.5, ax=ax) # plot camera frame
    
    t.append(np.matmul(T_we, np.matmul(T_camera_effector, T_cb))[:3,3])
    pass
print(np.std(t, axis=0))
ax.set_title('Reprojection from measured calibration')
ax.view_init(elev=20, azim=60)
plt.show()